<img src="https://3a2vzv37nkuw3esf6a3u2t7s-wpengine.netdna-ssl.com/webcentral/wp-content/uploads/sites/267/2019/02/horizontaldarkbackground.jpg" width="55%" align="left"> 
<br><br><br><br><br>
<font size=3 color='#08088A'>
<b>CSCI250 Python Computing: Building a Sensor System</b> <br><br>
Capstone final report
</font>
<hr style="height:5px" width="55%" align="left">

# Title

Insert one or more pictures of your project. 
You can do this by clicking **Edit** in the menu bar, then **Insert Image**.

# Group 

List all members and indicate specific project responsibilities:

* Tobin Ford, flex/pressure sensors, climbing, data analysis
* Alexander Capehart, accelerometers, climbing, data analysis

# Description

Our project a sensor system that can be attached to a climber to record biometrics during climbing.

# Hardware

- 5x Accelerometers
- 1x Qwiic Mux (To combine Accelerometers)
- 2x Pressure Sensors
- 2x Flex sensors

# Operation

- Attach Accelerometers to Mux, make sure to daisy chain jumpers so they span across all limbs
- Attach sensor system to climber, connect to RPi
- Run "data collection" part of this notebook when the climber starts climbing
- Specify the files to read in the "data analysis" part of this notebook

# Data

As stated in the capstone guidelines notebook, your project must have 4 data components:

1. **Collect** data
2. **Process** data
3. **Display** data
4. **Interpret** data



# Wiring

Include a a description or schematic of how all hardware elements are connected to your RPi. Be specific about what is connected where. There are several websites that allow you to create circuit schematics for free, such as [this one](https://www.circuit-diagram.org/editor/).

# Code

Include all the code that makes your project run. It can be in a single code cell or split between many cells, whatever you prefer. Add documentation to explain what different portions of your code do.

# Credits

Include references to all code you used in your project. If it's a library, provide name, version and download link.